In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, window, current_timestamp

# Create SparkSession
spark = SparkSession.builder \
    .appName("WordCountStreaming") \
    .getOrCreate()

# Define the schema for the streaming data
schema = "value STRING"

# Read streaming data from a TCP socket
lines = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"), current_timestamp().alias("timestamp"))

# Define a window of 10 seconds
windowedCounts = words \
    .groupBy(window(words.timestamp, "10 seconds"), words.word) \
    .count()

# Output the word counts to the console
query = windowedCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", False) \
    .start()



# Wait for the streaming to finish
query.awaitTermination()

# Stop the SparkSession
spark.stop()


24/05/12 22:43:48 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/05/12 22:43:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-093896e8-6217-4ba3-97d6-b9a106f7f821. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/05/12 22:43:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----+-----+
|window                                    |word|count|
+------------------------------------------+----+-----+
|{2024-05-12 22:44:00, 2024-05-12 22:44:10}|a   |1    |
+------------------------------------------+----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+----+-----+
|window                                    |word|count|
+------------------------------------------+----+-----+
|{2024-05-12 22:44:10, 2024-05-12 22:44:20}|b   |1    |
|{2024-05-12 22:44:00, 2024-05-12 22:44:10}|a   |1    |
+------------------------------------------+----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----+-----+
|window                                    |word|count|
+------------------------------------------+----+-----+
|{2024-05-12 22:44:30, 2024-05-12 22:44:40}|d   |1    |
|{2024-05-12 22:44:30, 2024-05-12 22:44:40}|c   |1    |
|{2024-05-12 22:44:30, 2024-05-12 22:44:40}|a   |2    |
|{2024-05-12 22:44:10, 2024-05-12 22:44:20}|b   |1    |
|{2024-05-12 22:44:00, 2024-05-12 22:44:10}|a   |1    |
|{2024-05-12 22:44:30, 2024-05-12 22:44:40}|b   |2    |
+------------------------------------------+----+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 